In [0]:
%python
df = spark.read.option("header", True).csv("/FileStore/tables/ipl_2021_matches.csv")


In [0]:
%python
df.createOrReplaceTempView("ipl_matches")


In [0]:
%python
df_matches = spark.read.option("header", True).csv("/FileStore/tables/ipl_2021_matches.csv")
df_matches.createOrReplaceTempView("ipl_matches")


In [0]:
%python
commentary = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/commentary.csv")
commentary.createOrReplaceTempView("commentary")


In [0]:
%python
commentary = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/commentary.csv")


In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/cell_phone.xlsx', name='cell_phone.xlsx', size=297905, modificationTime=1740663046000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-1.csv', name='cell_phones-1.csv', size=43927, modificationTime=1740667679000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-2.csv', name='cell_phones-2.csv', size=43927, modificationTime=1740667926000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-3.csv', name='cell_phones-3.csv', size=43927, modificationTime=1740717554000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-4.csv', name='cell_phones-4.csv', size=43927, modificationTime=1740851516000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones.csv', name='cell_phones.csv', size=43927, modificationTime=1740650761000),
 FileInfo(path='dbfs:/FileStore/tables/cleaned_ipl_2021/', name='cleaned_ipl_2021/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/combined_ipl_commentary.csv/', name='combined_ipl_commentary.csv/'

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession.builder.master("local").appName("IPL Analysis").getOrCreate()
data = [
    (2021, "CSK", "RCB", "completed", "CSK"),
    (2021, "MI", "KKR", "completed", "MI"),
    (2021, "SRH", "DC", "tied", None),
    (2022, "RR", "PBKS", "completed", "PBKS"),
    (2022, "GT", "LSG", "abandoned", None),
    (2023, "CSK", "MI", "completed", "MI"),
    (2023, "RCB", "RR", "completed", "RCB")
]
schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("match_team1", StringType(), True),
    StructField("match_team2", StringType(), True),
    StructField("result", StringType(), True),
    StructField("winner", StringType(), True),
])
ipl_df = spark.createDataFrame(data, schema)
ipl_df.createOrReplaceTempView("ipl")
ipl_df.show()


+----+-----------+-----------+---------+------+
|year|match_team1|match_team2|   result|winner|
+----+-----------+-----------+---------+------+
|2021|        CSK|        RCB|completed|   CSK|
|2021|         MI|        KKR|completed|    MI|
|2021|        SRH|         DC|     tied|  null|
|2022|         RR|       PBKS|completed|  PBKS|
|2022|         GT|        LSG|abandoned|  null|
|2023|        CSK|         MI|completed|    MI|
|2023|        RCB|         RR|completed|   RCB|
+----+-----------+-----------+---------+------+



In [0]:
%sql
SELECT 
  year, 
  COUNT(DISTINCT match_team1) + COUNT(DISTINCT match_team2) AS total_teams
FROM ipl
WHERE match_team1 IS NOT NULL AND match_team2 IS NOT NULL AND year IS NOT NULL
GROUP BY year
ORDER BY year;


year,total_teams
2021,6
2022,4
2023,4


In [0]:
%sql
SELECT 
  year,
  COUNT(CASE WHEN LOWER(result) LIKE '%abandoned%' THEN 1 END) AS abandoned,
  COUNT(CASE WHEN LOWER(result) LIKE '%tie%' THEN 1 END) AS tied,
  COUNT(CASE WHEN LOWER(result) LIKE '%completed%' THEN 1 END) AS completed
FROM ipl
GROUP BY year
ORDER BY year;


year,abandoned,tied,completed
2021,0,1,2
2022,1,0,1
2023,0,0,2


In [0]:
%sql
SELECT 
  year,
  ROUND(AVG(ball_count), 2) AS avg_balls_per_match
FROM (
  SELECT 
    c.match_id,
    i.year,
    COUNT(*) AS ball_count
  FROM commentary c
  JOIN ipl i ON c.match_id = i.match_no
  GROUP BY c.match_id, i.year
) sub
GROUP BY year
ORDER BY year;


year,avg_balls_per_match
2021,2.0
2022,1.0
2023,1.0


In [0]:
%sql
SELECT 
  year,
  winner AS team,
  COUNT(*) AS matches_won
FROM ipl
WHERE winner IS NOT NULL
GROUP BY year, winner
ORDER BY year, matches_won DESC;


year,team,matches_won
2021,CSK,1
2021,MI,1
2022,PBKS,1
2023,MI,1
2023,RCB,1
